In [2]:
import pandas as pd
from connection import connect
import numpy as np
from translate_language import convert_language


In [4]:
#connection
co_oltp,etl_conn,etl_conn_or=connect()


## Extract

In [5]:
#extract DimProductCategory


# from deep_translator import GoogleTranslator

dim_product=pd.read_sql_table('product',co_oltp,schema='production')
dim_product.columns


Index(['product_id', 'name', 'product_number', 'make_flag',
       'finished_goods_flag', 'color', 'safety_stock_level', 'reorder_point',
       'standard_cost', 'list_price', 'size', 'size_unit_measure_code',
       'weight_unit_measure_code', 'weight', 'days_to_manufacture',
       'product_line', 'class', 'style', 'product_subcategory_id',
       'product_model_id', 'sell_start_date', 'sell_end_date',
       'discontinued_date', 'rowguid', 'modified_date'],
      dtype='object')

## Transform


In [6]:

dim_product=dim_product.rename(columns={'product_number':'product_alternate_key','name':'english_product_name'})

dim_product.columns

dim_product_subcategory=pd.read_sql_query("""SELECT product_subcategory_key,product_subcategory_alternate_key
                                          FROM dim_product_subcategory 
                                         """,etl_conn)

dim_product= dim_product.merge(
    dim_product_subcategory[['product_subcategory_alternate_key','product_subcategory_key']],
    left_on='product_subcategory_id',
    right_on='product_subcategory_alternate_key',
    how='left'
)

dim_product.drop(['product_subcategory_alternate_key','product_subcategory_id'], axis=1 ,inplace=True)

dim_product.columns







Index(['product_id', 'english_product_name', 'product_alternate_key',
       'make_flag', 'finished_goods_flag', 'color', 'safety_stock_level',
       'reorder_point', 'standard_cost', 'list_price', 'size',
       'size_unit_measure_code', 'weight_unit_measure_code', 'weight',
       'days_to_manufacture', 'product_line', 'class', 'style',
       'product_model_id', 'sell_start_date', 'sell_end_date',
       'discontinued_date', 'rowguid', 'modified_date',
       'product_subcategory_key'],
      dtype='object')

In [13]:
# size_range, deal_price, model_name,large_photo,start_date,end_date,status

In [7]:
#size_range

def get_size_range(size):
    if not size:
        return 'NA'
    try:

        s = int(size)
        if 38 <= s <= 40:
            return '38–40 CM'
        elif 42 <= s <= 46:
            return '42–46 CM'
        elif 48 <= s <= 52:
            return '48–52 CM'
        elif 54 <= s <= 58:
            return '54–58 CM'
        elif 60 <= s <= 62:
            return '60–62 CM'
        else:
            return 'NA'
    except ValueError:
        pass
    
dim_product['size_range']=dim_product['size'].apply(get_size_range)   

# dim_product[dim_product['product_alternate_key']=='FR-M94S-44']



In [8]:
#deal_price

t_sales_order_d = pd.read_sql("""
   
    SELECT 
    p.product_id,
    
    p.standard_cost,
    
    ROUND(AVG(sod.unit_price * (1 - sod.unit_price_discount)), 4) AS dealer_price
    FROM sales.sales_order_detail AS sod
    JOIN sales.sales_order_header AS soh 
        ON sod.sales_order_id = soh.sales_order_id
    JOIN sales.customer AS c 
        ON soh.customer_id = c.customer_id
    JOIN sales.store AS s 
        ON c.store_id = s.business_entity_id
    JOIN production.product AS p 
        ON sod.product_id = p.product_id
    GROUP BY p.product_id, p.standard_cost
    ORDER BY p.product_id;


    """, co_oltp)

t_sales_order_d

dim_product=dim_product.merge(
    t_sales_order_d[['product_id','dealer_price']],
    on='product_id',
    how='left'

)






In [9]:
#model name

t_product_model=pd.read_sql("SELECT product_model_id, name FROM production.product_model",co_oltp)

dim_product=dim_product.merge(
    t_product_model,
    on='product_model_id',
    how='left'
).rename(columns={'name':'model_name'})

dim_product

,product_id,english_product_name,product_alternate_key,make_flag,finished_goods_flag,color,safety_stock_level,reorder_point,standard_cost,list_price,...,product_model_id,sell_start_date,sell_end_date,discontinued_date,rowguid,modified_date,product_subcategory_key,size_range,dealer_price,model_name
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,96.0,2013-05-30 00:00:00+00:00,NaT,NaT,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827000+00:00,5.0,NA,NaN,ML Bottom Bracket
500,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,97.0,2013-05-30 00:00:00+00:00,NaT,NaT,230C47C5-08B2-4CE3-B706-69C0BDD62965,2014-02-08 10:01:36.827000+00:00,5.0,NA,72.8940,HL Bottom Bracket
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,NaT,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827000+00:00,2.0,42–46 CM,323.9940,Road-750
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,NaT,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827000+00:00,2.0,48–52 CM,323.1322,Road-750


In [10]:
#large_photo

t_large_photo= pd.read_sql("""SELECT ppp.product_id, pp.large_photo
                           
                           FROM production.product_photo AS pp, production.product_product_photo AS ppp

                           WHERE pp.product_photo_id=ppp.product_photo_id

                           
                           """,co_oltp)


dim_product=dim_product.merge(
    t_large_photo,
    on='product_id',
    how='left'
)

# dim_product

In [11]:
#start_date,end_date PREGUNTAR!


t_product_price_list_history=pd.read_sql("""SELECT ph.product_id, ph.start_date, ph.end_date
                                         FROM production.product_list_price_history as ph   
                                         """,co_oltp)

dim_product=dim_product.merge(
    t_product_price_list_history,
    on='product_id',
    how='left'
    
)



# t_product_price_list_history

# dim_product


In [12]:
#satus --> this depend of end_date

dim_product['status']=dim_product['end_date'].apply(lambda x: 'Current' if pd.isna(x) else None )


In [ ]:
# description languages
from transformers import MarianMTModel, MarianTokenizer

t_language_description=pd.read_sql("""
    SELECT p.product_id,  pd.description ,pc.name
    FROM production.product as p JOIN production.product_model_prod_desc_culture as pmpdc
    ON p.product_model_id=pmpdc.product_model_id
    JOIN production.product_description as pd
    ON pmpdc.product_description_id=pd.product_description_id
    JOIN production.culture as pc
    ON pmpdc.culture_id=pc.culture_id
""",co_oltp)


columns=t_language_description['name'].unique().tolist()

new_columns={name:f'{name.lower()}_description' for name in columns}


t_language_description=t_language_description.pivot(index='product_id',columns='name',values='description').reset_index()

t_language_description.rename(columns=new_columns,inplace=True)


dim_product=dim_product.merge(
    t_language_description,
    on='product_id',
    how='left'
)


# def convert_language(src_lang,tgt_lang,src_name,tgt_name,df:pd.DataFrame):

#     model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'

#     tokenizer = MarianTokenizer.from_pretrained(model_name)
#     model = MarianMTModel.from_pretrained(model_name)

#     def translate_batch(texts):
#         inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
#         translated = model.generate(**inputs)
#         return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]


#     mask=df[src_name].notna()
#     texts=df.loc[mask,src_name].tolist()

#     batch_size=50

#     translations=[]

#     for i in range(0,len(texts),batch_size):
#         batch=texts[i:i+batch_size]
#         translations.extend(translate_batch(batch))

#     df.loc[mask,tgt_name]=translations

#     return df


dim_product=convert_language('en','fr','english_product_name','french_product_name',dim_product)
dim_product=convert_language('en','es','english_product_name','spanish_product_name',dim_product)
dim_product=convert_language('en','jap','english_description','japanese_description',dim_product)
dim_product=convert_language('en','de','english_description','german_description',dim_product)
dim_product=convert_language('en','trk','english_description','turkish_description',dim_product)

dim_product










--- Loading model Helsinki-NLP/opus-mt-en-fr (this should only happen once) ---


c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 606 total rows, but only 504 unique values to translate.
Translation complete.
--- Loading model Helsinki-NLP/opus-mt-en-es (this should only happen once) ---


c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 606 total rows, but only 504 unique values to translate.
Translation complete.
--- Loading model Helsinki-NLP/opus-mt-en-jap (this should only happen once) ---


...

c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 606 total rows, but only 114 unique values to translate.
Translation complete.
--- Loading model Helsinki-NLP/opus-mt-en-trk (this should only happen once) ---


c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 606 total rows, but only 114 unique values to translate.
Translation complete.


,product_id,english_product_name,product_alternate_key,make_flag,finished_goods_flag,color,safety_stock_level,reorder_point,standard_cost,list_price,...,chinese_description,english_description,french_description,hebrew_description,thai_description,french_product_name,spanish_product_name,japanese_description,german_description,turkish_description
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,Course réglable,Carrera ajustable,NaN,NaN,NaN
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,Balle de roulement,Rodamiento de bolas,NaN,NaN,NaN
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,BB Roulement à billes,Rodamiento de bolas BB,NaN,NaN,NaN
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,Roulements à billes pour casques,Rodamientos de bolas de auriculares,NaN,NaN,NaN
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,Lame,Hoja,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,铝合金车圈；大直径脚蹬轴。,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่,ML Bracket inférieur,ML Bracket inferior,"種々 の 工作 を し て い る , 保証 石 を お き なさ い .",Aluminium-Legierung Tassen; große Spindel Durc...,Алтынум алминийлық полюс; чоң диаметриялы рель...
602,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,铝合金车圈和空心轴。,Aluminum alloy cups and a hollow axle.,Cuvettes en alliage d'aluminium et axe creux.,כיסויים מסגסוגת אלומיניום וציר חלול.,ดุมอลูมิเนียมอัลลอยด์และเพลากลวง,Poignée de fond HL,Soporte inferior HL,"ここで,うすずきを,すず findと,うずめにしてつけなさい.",Aluminium-Legierung Tassen und eine Hohlachse.,"Альумум мага магарестрскалар, массажлы томанна..."
603,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,"Route 750 Noir, 44","Road-750 Black, 44","ザカン を 打 つ 者 , ボザン を 打 つ 者 , ナアラ を 倒 す 者 を 倒 す ...",Einsteiger-Rad für Erwachsene; bietet eine kom...,"Uşak bisiklete; rahat участогna, lehurrea ýa b..."
604,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,"Route 750 Noir, 48","Road-750 Black, 48","ザカン を 打 つ 者 , ボザン を 打 つ 者 , ナアラ を 倒 す 者 を 倒 す ...",Einsteiger-Rad für Erwachsene; bietet eine kom...,"Uşak bisiklete; rahat участогna, lehurrea ýa b..."


In [14]:
# dim_product_olap=pd.read_sql_table('dim_product',etl_conn_or)
# columns_olap=set(dim_product_olap.columns)

# columns_oltp=set(dim_product.columns)

# columns_olap-columns_oltp


dim_product=dim_product.drop(['product_id','make_flag','product_model_id','discontinued_date','rowguid','modified_date','sell_start_date','sell_end_date'],axis=1)









In [15]:
# convert null to 'NA' in color and replace 'nan' objects

# dim_product_olap['color'].unique()

dim_product['color']=dim_product['color'].apply(lambda x:'NA' if pd.isna(x) else x)
dim_product = dim_product.replace({'nan':None})

dim_product

,english_product_name,product_alternate_key,finished_goods_flag,color,safety_stock_level,reorder_point,standard_cost,list_price,size,size_unit_measure_code,...,chinese_description,english_description,french_description,hebrew_description,thai_description,french_product_name,spanish_product_name,japanese_description,german_description,turkish_description
0,Adjustable Race,AR-5381,False,NA,1000,750,0.0000,0.00,None,None,...,NaN,NaN,NaN,NaN,NaN,Course réglable,Carrera ajustable,NaN,NaN,NaN
1,Bearing Ball,BA-8327,False,NA,1000,750,0.0000,0.00,None,None,...,NaN,NaN,NaN,NaN,NaN,Balle de roulement,Rodamiento de bolas,NaN,NaN,NaN
2,BB Ball Bearing,BE-2349,False,NA,800,600,0.0000,0.00,None,None,...,NaN,NaN,NaN,NaN,NaN,BB Roulement à billes,Rodamiento de bolas BB,NaN,NaN,NaN
3,Headset Ball Bearings,BE-2908,False,NA,800,600,0.0000,0.00,None,None,...,NaN,NaN,NaN,NaN,NaN,Roulements à billes pour casques,Rodamientos de bolas de auriculares,NaN,NaN,NaN
4,Blade,BL-2036,False,NA,800,600,0.0000,0.00,None,None,...,NaN,NaN,NaN,NaN,NaN,Lame,Hoja,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,ML Bottom Bracket,BB-8107,True,NA,500,375,44.9506,101.24,None,None,...,铝合金车圈；大直径脚蹬轴。,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่,ML Bracket inférieur,ML Bracket inferior,"種々 の 工作 を し て い る , 保証 石 を お き なさ い .",Aluminium-Legierung Tassen; große Spindel Durc...,Алтынум алминийлық полюс; чоң диаметриялы рель...
602,HL Bottom Bracket,BB-9108,True,NA,500,375,53.9416,121.49,None,None,...,铝合金车圈和空心轴。,Aluminum alloy cups and a hollow axle.,Cuvettes en alliage d'aluminium et axe creux.,כיסויים מסגסוגת אלומיניום וציר חלול.,ดุมอลูมิเนียมอัลลอยด์และเพลากลวง,Poignée de fond HL,Soporte inferior HL,"ここで,うすずきを,すず findと,うずめにしてつけなさい.",Aluminium-Legierung Tassen und eine Hohlachse.,"Альумум мага магарестрскалар, массажлы томанна..."
603,"Road-750 Black, 44",BK-R19B-44,True,Black,100,75,343.6496,539.99,44,CM,...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,"Route 750 Noir, 44","Road-750 Black, 44","ザカン を 打 つ 者 , ボザン を 打 つ 者 , ナアラ を 倒 す 者 を 倒 す ...",Einsteiger-Rad für Erwachsene; bietet eine kom...,"Uşak bisiklete; rahat участогna, lehurrea ýa b..."
604,"Road-750 Black, 48",BK-R19B-48,True,Black,100,75,343.6496,539.99,48,CM,...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...,"Route 750 Noir, 48","Road-750 Black, 48","ザカン を 打 つ 者 , ボザン を 打 つ 者 , ナアラ を 倒 す 者 を 倒 す ...",Einsteiger-Rad für Erwachsene; bietet eine kom...,"Uşak bisiklete; rahat участогna, lehurrea ýa b..."


## Load

In [64]:
dim_product.to_sql('dim_product', etl_conn, if_exists='append',index=False)

606